# Process bulk profiles

## Import libraries

In [1]:
import pathlib
import pprint

import pandas as pd

from pycytominer import aggregate, annotate, normalize, feature_select
from pycytominer.cyto_utils import output

## Set paths and variables

In [2]:
# Condition for how to normalize the plates
normalize_with_U2OS = (
    False  # Set to False if normalizing to whole plate versus just `U2-OS` cell line
)

# Set round to be processed
round_id = "Round_2_data"

# Path to dir with cleaned data from single-cell QC
cleaned_dir = pathlib.Path(f"./data/cleaned_profiles/{round_id}")

# output path for bulk profiles
output_dir = pathlib.Path(f"./data/bulk_profiles/{round_id}")
output_dir.mkdir(parents=True, exist_ok=True)

# extract the plate names from the file name
plate_names = [file.stem.split("_")[0] for file in cleaned_dir.glob("*.parquet")]

# path for platemap directory
platemap_dir = pathlib.Path("../0.download_data/metadata/platemaps")

# load in barcode platemap
barcode_platemap = pd.read_csv(
    pathlib.Path(f"{platemap_dir}/Barcode_platemap_pilot_data.csv")
)

# operations to perform for feature selection
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

plate_names

['BR00145438',
 'BR00145818',
 'BR00145439',
 'BR00145816',
 'BR00145440',
 'BR00145817']

## Set dictionary with plates to process

In [3]:
# create plate info dictionary
plate_info_dictionary = {
    name: {
        "profile_path": (
            str(
                pathlib.Path(
                    list(cleaned_dir.rglob(f"{name}_cleaned.parquet"))[0]
                ).resolve(strict=True)
            )
            if list(cleaned_dir.rglob(f"{name}_cleaned.parquet"))
            else None
        ),
        # Find the platemap file based on barcode match and append .csv
        "platemap_path": (
            str(
                pathlib.Path(
                    list(
                        platemap_dir.rglob(
                            f"{barcode_platemap.loc[barcode_platemap['barcode'] == name, 'platemap_file'].values[0]}.csv"
                        )
                    )[0]
                ).resolve(strict=True)
            )
            if name in barcode_platemap["barcode"].values
            else None
        ),
        # Get the time_point based on the barcode match
        "time_point": (
            barcode_platemap.loc[
                barcode_platemap["barcode"] == name, "time_point"
            ].values[0]
            if name in barcode_platemap["barcode"].values
            else None
        ),
    }
    for name in plate_names
}

# Display the dictionary to verify the entries
pprint.pprint(plate_info_dictionary, indent=4)

{   'BR00145438': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate3_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profiles/Round_2_data/BR00145438_cleaned.parquet',
                      'time_point': 24},
    'BR00145439': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate3_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profiles/Round_2_data/BR00145439_cleaned.parquet',
                      'time_point': 48},
    'BR00145440': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate3_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profil

## Process data with pycytominer

In [4]:
# Determine suffix based on normalize_with_U2OS
u2os_suffix = "_U2OS_samples" if normalize_with_U2OS else ""

# If normalizing with U2-OS, create a subfolder named 'U2OS_samples'
if normalize_with_U2OS:
    U2OS_output_dir = output_dir / "U2OS_samples"
    U2OS_output_dir.mkdir(exist_ok=True)
else:
    U2OS_output_dir = (
        output_dir  # Otherwise, use the root output_dir for whole plate normalization
    )

for plate, info in plate_info_dictionary.items():
    print(f"Now performing pycytominer pipeline for {plate}")

    # Output file paths for each file
    output_aggregated_file = str(pathlib.Path(f"{output_dir}/{plate}_bulk.parquet"))
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_annotated.parquet")
    )

    # Save normalized and feature-selected files in U2OS_samples folder if needed
    output_normalized_file = str(
        pathlib.Path(f"{U2OS_output_dir}/{plate}_bulk_normalized{u2os_suffix}.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(
            f"{U2OS_output_dir}/{plate}_bulk_feature_selected{u2os_suffix}.parquet"
        )
    )

    # Load single-cell profiles
    single_cell_df = pd.read_parquet(info["profile_path"])

    # Load platemap
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Aggregation
    aggregate(
        population_df=single_cell_df,
        operation="median",
        strata=["Image_Metadata_Plate", "Image_Metadata_Well"],
        output_file=output_aggregated_file,
        output_type="parquet",
    )

    # Step 2: Annotation
    annotated_df = annotate(
        profiles=output_aggregated_file,
        platemap=platemap_df,
        join_on=["Metadata_well", "Image_Metadata_Well"],
    )

    # Step 2.5: Add 'Metadata_time_point' column based on the plate's time_point from dict
    annotated_df["Metadata_time_point"] = info["time_point"]

    # Step 3: Output annotated DataFrame
    output(
        df=annotated_df,
        output_filename=output_annotated_file,
        output_type="parquet",
    )

    # Step 4: Normalization
    samples = (
        "Metadata_cell_line == 'U2-OS'" if normalize_with_U2OS else "all"
    )  # "all" is the default to perform on whole plate
    print(f"Normalizing using samples: {samples}")

    normalize(
        profiles=annotated_df,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
        samples=samples,
    )

    # Step 5: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        output_file=output_feature_select_file,
        output_type="parquet",
    )

Now performing pycytominer pipeline for BR00145438
Normalizing using samples: all
Now performing pycytominer pipeline for BR00145818
Normalizing using samples: all
Now performing pycytominer pipeline for BR00145439
Normalizing using samples: all
Now performing pycytominer pipeline for BR00145816
Normalizing using samples: all
Now performing pycytominer pipeline for BR00145440
Normalizing using samples: all
Now performing pycytominer pipeline for BR00145817
Normalizing using samples: all


In [5]:
# Check output file
test_df = pd.read_parquet(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(99, 671)


,Metadata_cell_line,Metadata_row,Metadata_column,Metadata_seeding_density,Metadata_condition,Metadata_Plate,Metadata_Well,Metadata_time_point,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,...,Nuclei_Texture_Correlation_CorrDNA_3_03_256,Nuclei_Texture_Correlation_CorrER_3_01_256,Nuclei_Texture_Correlation_CorrRNA_3_00_256,Nuclei_Texture_Correlation_CorrRNA_3_01_256,Nuclei_Texture_Correlation_CorrRNA_3_02_256,Nuclei_Texture_Correlation_CorrRNA_3_03_256,Nuclei_Texture_Entropy_CorrDNA_3_01_256,Nuclei_Texture_InfoMeas1_CorrMito_3_00_256,Nuclei_Texture_SumVariance_CorrBrightfield_3_02_256,Nuclei_Texture_SumVariance_CorrER_3_03_256
0,CHLA-10,B,3,1000,synthemax,BR00145817,B03,48,-0.777717,0.739026,...,-0.172024,0.578486,0.372921,0.640877,0.980610,0.606146,0.118309,-0.150405,-0.634792,-0.481880
1,CHLA-10,C,3,1000,synthemax,BR00145817,C03,48,-0.671346,0.606786,...,-0.018912,0.357497,0.496653,0.882495,0.615007,0.638179,0.408401,0.173001,-0.589223,-0.618181
